In [ ]:
import pandas as pd
import requests
import time

In [ ]:
count = 1
ID_list = []
while(count <= 150):
  url = f"https://www.99acres.com/api-aggregator/srp/search?property_type=1&area_unit=1&platform=DESKTOP&moduleName=GRAILS_SRP&workflow=GRAILS_SRP&page_size=30&page={count}&city=25&preference=S&res_com=R&seoUrlType=DEFAULT"
  response = requests.get(url)
  response = response.json()
  if response['status']['code'] != 1:
      break
  else:
    ids = response['properties']
    c = 1
    for id in ids:
      if c != 31:
        ID_list.append(id['PROP_ID'])
        c += 1
  count += 1

print(len(ID_list))

4500


In [ ]:
output = pd.DataFrame()

for id in ID_list:
  try:
    url = f"https://www.99acres.com/api-aggregator/psuedoData?prop_id={id}&stage=load"
    # time.sleep(0.4)
    response = requests.get(url)
    response = response.json()

    prop_data = response['propertyDetails']['prop_data']

    lattitude = prop_data['Latitude']
    longitude = prop_data['Longitude']
    post_date = prop_data['Modify_Date']
    availability_date = prop_data['Availability_Info']['Availability_Date']
    owner_name = prop_data['company_label']
    property_name = prop_data['Prop_Name']
    id = prop_data['Building_Id']
    locality_name = prop_data ['Locality_Name']
    base_price = prop_data['Price_Per_Unit_Area']

    for floor in response['floorPlan']:
      area_sqft = floor['SALEABLE_AREA_SQFT']
      price = floor['NEW_BOOKING_PRICE']['min']
      details = floor['unitDetailSectionParts']
      bedrooms = details['bedrooms']
      kitchens = details['kitchens']
      bathroom = details['bathroom']
      balcony = details['balcony']
      living = details['living']
      data = {'id': id, 
              'lattitude': lattitude, 
              'longitude': longitude, 
              'post_date': post_date, 
              'availability_date': availability_date, 
              'owner_name': owner_name, 
              'property_name': property_name, 
              'locality_name': locality_name, 
              'base_price': base_price, 
              'area_sqft': area_sqft, 
              'price': price, 
              'bedrooms': bedrooms, 
              'kitchens': kitchens, 
              'bathroom': bathroom, 
              'balcony': balcony, 
              'living': living}
      output = output.append(data, ignore_index=True)
  except:
    pass

In [ ]:
output.to_csv(r'Datasets\raw_dataset.csv')